In [ ]:
# Import required modules
import pandas as pd
import re                                           # regular expressions
import nltk
import nltk.corpus as corpus                        # nltk package for reading files
nltk.download('stopwords')
from nltk.corpus import stopwords as stopwords      # words such as 'the','be','an', that don't significantly impact the sentiment
from nltk.stem import WordNetLemmatizer             # Groups words
from nltk.tokenize import word_tokenize             # Splits text into tokens
import collections
# Will be used later on
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
# import sklearn
# import numpy as np

In [ ]:
# Filter out all unlabelled sentiment
p_df = pd.read_csv('/home/mia/Documents/College/CS4811/TwitterNLP/tweets_labelled.csv', sep=';',names=['created_at','text','sentiment']).query('sentiment == "positive"')
n_df = pd.read_csv('/home/mia/Documents/College/CS4811/TwitterNLP/tweets_labelled.csv', sep=';',names=['created_at','text','sentiment']).query('sentiment == "negative"')

# Debug Print
# display(p_df)
# display(n_df)

# Combine dataframes
frames = [p_df, n_df]
df = pd.concat(frames)

# Sort by date
# https://www.geeksforgeeks.org/how-to-sort-a-pandas-dataframe-by-date/
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.sort_values(by="created_at",ascending=True)
# Debug Print
display(df)


In [41]:
# Grab the top 25 stocks
ticker_pattern = re.compile(r'(^\$[A-Z]+|^\$ES_F)')

# Break the string into a dictionary
ticker_freq_dict = collections.defaultdict(int)

for text in df['text']:
    for word in text.split():
        if ticker_pattern.fullmatch(word) is not None:
            ticker_freq_dict[word[1:]] += 1

ticker_freq_df = pd.DataFrame.from_dict(
    ticker_freq_dict, orient='index', columns=['freq'])\
    .sort_values('freq', ascending=False).head(25)
display(ticker_freq_df)    

,freq
SPX,132
AMZN,80
AAPL,79
SPY,69
FB,59
TSLA,40
MSFT,32
QQQ,29
NFLX,27
DIS,22


In [44]:
# Create a dictionary of strings to save each within the dictionary
ticker_dict = collections.defaultdict(str)

for text in df['text']:
    for ticker in ticker_freq_df.index:
        if text.find(ticker) > 0:
            if ticker not in ticker_dict:
                ticker_dict[ticker] = [text]
                continue
            else:
                ticker_dict[ticker].append(text)
                continue
        # else:
        #     ticker_dict[ticker].append(text)

# Confirm the correct amount of tweets have been written to the file
for ticker in ticker_dict:
    if len(ticker_dict[ticker]) != ticker_freq_dict[ticker]:
        print("ERROR: " + ticker + " expected: " + str(ticker_freq_dict[ticker]) + ", Actual: " + str(len(ticker_dict[ticker])))
    

ERROR: AMZN expected: 80, Actual: 85
ERROR: FB expected: 59, Actual: 70
ERROR: TSLA expected: 40, Actual: 42
ERROR: ZM expected: 15, Actual: 12
ERROR: T expected: 15, Actual: 638
ERROR: SPX expected: 132, Actual: 137
ERROR: IWM expected: 15, Actual: 17
ERROR: NDX expected: 15, Actual: 16
ERROR: QQQ expected: 29, Actual: 33
ERROR: AAPL expected: 79, Actual: 88
ERROR: NFLX expected: 27, Actual: 29
ERROR: BA expected: 17, Actual: 41
ERROR: GOOGL expected: 16, Actual: 17
ERROR: GOOG expected: 15, Actual: 32
ERROR: BAC expected: 13, Actual: 15
ERROR: SPY expected: 69, Actual: 71
ERROR: JPM expected: 22, Actual: 23
ERROR: ES_F expected: 20, Actual: 29
ERROR: DIS expected: 22, Actual: 24
ERROR: INTC expected: 11, Actual: 10
ERROR: MSFT expected: 32, Actual: 36


In [ ]:
# Print out the dictionary to json file for debugging purposes
import json

with open('ticker_dict.json','w') as fout:
    json.dump(ticker_dict, fout, indent=4)

In [ ]:
# Return a weight from 0-1 based on the percentage of positive tweets
